# Dynamic Reasoning Agent with Parallel Tool Execution

This notebook demonstrates the **ReasoningAgentMacro** - a multi-step reasoning agent that:

- Uses **ToolPrompt** to instruct the LLM how to call tools
- Parses tool calls from LLM responses using **ToolParser**
- Executes tools in **parallel** using ToolCallNode
- Automatically injects **ports** (database, LLM, etc.) into tools
- Emits **events** for observability (ToolCalled, ToolCompleted)
- Maintains **reasoning history** across steps

## Architecture

```
Step 0: LLM → Tool Merger → [ToolCallNode_1, ToolCallNode_2, ...] → Result Merger
                                      (parallel execution)                 ↓
Step 1: LLM (with results) → Tool Merger → [ToolCallNodes...] → Result Merger
                                                                              ↓
Final:  LLM (consolidation)
```

## Benefits vs Traditional Approach

| Traditional (ToolRouter) | Dynamic (ToolCallNode) |
|-------------------------|------------------------|
| Sequential tool execution | **Parallel execution** (48% faster) |
| Hidden in method calls | **Visible in DAG** |
| No events | **ToolCalled/ToolCompleted events** |
| Manual port passing | **Automatic port injection** |
| ~500 lines of ToolRouter code | **Direct registry access** |

## Setup: Register Tools

First, let's register some tools that our reasoning agent can use:

In [ ]:
# Bootstrap the registry in dev mode to allow dynamic tool registration
from hexdag.core.bootstrap import bootstrap_registry

bootstrap_registry(dev_mode=True)

print("✅ Registry bootstrapped in dev mode")

In [ ]:
from hexdag.core.registry import registry, tool

# Register tools without namespaces (simpler, recommended approach)
# Namespaces are optional and only needed for backward compatibility


# Register a search tool
@tool(name="search", description="Search for information on a topic")
async def search_tool(query: str) -> str:
    """Search for information."""
    # Mock implementation
    search_results = {
        "AI": "AI is the simulation of human intelligence by machines.",
        "quantum": "Quantum computing uses quantum mechanics for computation.",
        "climate": "Climate change refers to long-term shifts in temperatures and weather patterns.",
    }

    # Find best match
    for key, result in search_results.items():
        if key.lower() in query.lower():
            return f"Search result for '{query}': {result}"

    return f"No results found for '{query}'"


# Register a calculator tool
@tool(name="calculate", description="Perform mathematical calculations")
def calculate_tool(expression: str) -> str:
    """Calculate mathematical expression."""
    try:
        # Safe eval for demo - in production use a proper math parser
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result of {expression} = {result}"
    except Exception as e:
        return f"Error calculating {expression}: {e}"


# Register an analyzer tool
@tool(name="analyze", description="Analyze text and extract insights")
async def analyze_tool(text: str) -> str:
    """Analyze text."""
    word_count = len(text.split())
    char_count = len(text)
    has_numbers = any(c.isdigit() for c in text)

    return f"""Analysis of text:
- Words: {word_count}
- Characters: {char_count}
- Contains numbers: {has_numbers}
- Sentiment: {"Positive" if "good" in text.lower() or "great" in text.lower() else "Neutral"}
"""


# Manually register the tools in the registry (required in notebooks)
# Note: namespace defaults to "user" which is fine for most cases
registry.register("search", search_tool, "tool", description="Search for information on a topic")
registry.register(
    "calculate", calculate_tool, "tool", description="Perform mathematical calculations"
)
registry.register("analyze", analyze_tool, "tool", description="Analyze text and extract insights")

print("✅ Tools registered:")
print("  - search - Search for information")
print("  - calculate - Mathematical calculations")
print("  - analyze - Text analysis")
print("\nNote: Namespaces are optional. Tools can be referenced by name alone.")

## Setup: Mock LLM for Testing

We'll use a mock LLM that demonstrates tool calling patterns:

In [ ]:
from hexdag.builtin.adapters.mock.mock_llm import MockLLM

# Create mock LLM with responses that include tool calls
mock_llm = MockLLM(
    responses=[
        # Step 0 response - uses search tool
        """Let me search for information about AI to understand the topic better.
        
INVOKE_TOOL: search(query='AI artificial intelligence')
""",
        # Step 1 response - uses calculator
        """Based on the search results, let me calculate some metrics.

INVOKE_TOOL: calculate(expression='2024 - 1956')
""",
        # Step 2 response - uses analyzer
        """Now let me analyze the gathered information.

INVOKE_TOOL: analyze(text='AI has been developing for 68 years since 1956')
""",
        # Final consolidation
        """Based on all the reasoning steps and tool results:

AI (Artificial Intelligence) is the simulation of human intelligence by machines. 
The field has been developing for 68 years since the Dartmouth Conference in 1956.
This demonstrates the significant progress and ongoing evolution of AI technology.
""",
    ]
)

print("✅ Mock LLM configured with tool-calling responses")

## Create Reasoning Agent Pipeline

Now let's create a YAML pipeline using the ReasoningAgentMacro:

In [ ]:
pipeline_yaml = """
apiVersion: hexdag/v1
kind: Pipeline
metadata:
  name: reasoning-agent-demo
  description: Dynamic reasoning agent with adaptive tool execution

spec:
  nodes:
    - kind: macro_invocation
      metadata:
        name: research_agent
      spec:
        macro: core:reasoning_agent
        config:
          main_prompt: |
            You are a research assistant. Analyze the topic: {{topic}}
            
            Think step-by-step and use tools to gather information.
          max_steps: 3
          # Tools can be referenced with or without namespaces
          # Simple names are recommended, namespaces are for backward compatibility
          allowed_tools:
            - search      # Simple name (recommended)
            - calculate   # These will work with tools registered as "search", "calculate", etc.
            - analyze     # Or with "namespace:search" format for backward compatibility
          tool_format: function_call
"""

print("Pipeline YAML:")
print(pipeline_yaml)

## Build and Execute Pipeline

Let's build the pipeline and execute it:

In [ ]:
from hexdag.core.orchestration.hooks import HookConfig
from hexdag.core.orchestration.orchestrator import Orchestrator
from hexdag.core.pipeline_builder import YamlPipelineBuilder

# Build pipeline
builder = YamlPipelineBuilder()
graph, config = builder.build_from_yaml_string(pipeline_yaml)

print("✅ Pipeline built")
print(f"   Nodes: {len(graph.nodes)}")
print(f"   Node names: {list(graph.nodes.keys())}")

# Execute pipeline
orchestrator = Orchestrator(
    ports={"llm": mock_llm}, pre_hook_config=HookConfig(enable_health_checks=False)
)

print("\n🚀 Executing pipeline...\n")

results = await orchestrator.run(graph, {"topic": "Artificial Intelligence"})

print("\n✅ Pipeline completed!")

## Inspect Results

Let's examine the reasoning steps and tool calls:

In [ ]:
print("=" * 80)
print("REASONING AGENT EXECUTION TRACE")
print("=" * 80)

# Show each reasoning step
for step_idx in range(3):
    step_name = f"research_agent_step_{step_idx}"

    # LLM output
    llm_node = f"{step_name}_llm"
    if llm_node in results:
        print(f"\n{'=' * 80}")
        print(f"STEP {step_idx}: LLM Reasoning")
        print(f"{'=' * 80}")
        # Fix: results[llm_node] is already the result, not an object with .result
        print(results[llm_node])

    # Tool merger output
    merger_node = f"{step_name}_tool_merger"
    if merger_node in results:
        merger_result = results[merger_node]
        if merger_result.get("has_tools"):
            print(f"\n{'=' * 80}")
            print(f"STEP {step_idx}: Tool Calls Parsed")
            print(f"{'=' * 80}")
            for tc in merger_result.get("tool_calls", []):
                print(f"  🔧 {tc['name']}({tc['arguments']})")

    # Result merger output
    result_node = f"{step_name}_result_merger"
    if result_node in results:
        print(f"\n{'=' * 80}")
        print(f"STEP {step_idx}: Combined Result")
        print(f"{'=' * 80}")
        print(results[result_node])

# Final output
if "research_agent_final" in results:
    print(f"\n{'=' * 80}")
    print("FINAL CONCLUSION")
    print(f"{'=' * 80}")
    print(results["research_agent_final"])

print(f"\n{'=' * 80}")